In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


In [2]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0,1])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)


ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)


dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f6b6c0facc0>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

In [4]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



8.66740439897978e-05
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐                                                                                                                                                                                                                       ┌──┐   ┌──┐                                                                                                                                                                                                                             ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@──

In [5]:
len(simplified_db), len(circuit_db)

(132, 240)